In [1]:
# Here I take inputs from the user and make a prediction, based on the models created in 
# the Main notebook

In [2]:
import pickle
import pandas as pd
import regex as re
import nltk 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to C:\Users\Diogo
[nltk_data]     Gonçalves\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Diogo Gonçalves\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Diogo
[nltk_data]     Gonçalves\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Diogo
[nltk_data]     Gonçalves\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# list of valid movie genres
valid_genres = ['action','adventure','animation','biography','comedy','crime','documentary',
          'drama','family','fantasy','history','horror','music','musical','mystery',
          'news','romance','sci-fi','sport','thriller','war', 'western']

valid_ratings = ['Approved','G','NC-17','Not Rated','PG','PG-13','R','TV-14','TV-G','TV-MA','TV-PG',
                 'Unrated','X']

In [4]:
#load movies
#movies = pickle.load(open('moviesROI.pkl',"rb"))

In [5]:
# Getting list of valid actors
#valid_actors = set(movies.actor1.unique())
#valid_actors = valid_actors | set(movies.actor2.unique())
#valid_actors = valid_actors | set(movies.actor3.unique())
#valid_actors = list(valid_actors)[1:]

In [6]:
# Getting list of valid directors
#valid_directors = list(set(movies.directors.unique()))

In [7]:
# importing table to translate ids to names
# name.basics.tsv.gz – Contains the following information for names:
# nconst (string) - alphanumeric unique identifier of the name/person
# primaryName (string)– name by which the person is most often credited

#path = 'Data/'
#names = pickle.load(open(path+"name.basics.sav","rb"))
#names = names[['nconst', 'primaryName']]

In [8]:
# adding primary name to valid directors and valid actors

# creating dfs
#actors = pd.DataFrame(valid_actors, columns=['actorID'])
#directors = pd.DataFrame(valid_directors, columns=['directorID'])

In [9]:
# merging name to dfs
#actors = pd.merge(actors,names,left_on='actorID',right_on='nconst')
#directors = pd.merge(directors,names,left_on='directorID',right_on='nconst')

In [10]:
# drop useless columns
#actors.drop(columns='actorID',inplace=True)
#directors.drop(columns='directorID',inplace=True)

In [11]:
# populating validation lists with names instead of ids
#valid_actors = actors['primaryName'].to_list()
#valid_directors = directors['primaryName'].to_list()

In [12]:
# save the resulting tables so I don't have to run this every time
#pickle.dump(valid_actors,open('valid_actors.pkl',"wb"))
#pickle.dump(valid_directors,open('valid_directors.pkl',"wb"))

In [13]:
# load saved tables
valid_actors = pickle.load(open('valid_actors.pkl',"rb"))
valid_directors = pickle.load(open('valid_directors.pkl',"rb"))

In [14]:
def get_inputs():
    '''
    returns a dictionary with the user inputs
    '''

    # year
    def valid_year(year):
        try:
            year = int(year)
            if year >= 1980 and year < 2025:
                return True
        except:
            return False

    year = input("Please enter the year your movie will come out :\n")
    while not valid_year(year):
        print('Please enter a valid year between 1980 and 2025.')
        year = input("Please enter the year your movie will come out :\n")

    year = int(year)

    # runtimeMinutes
    def valid_runtimeMinutes(minutes):
        try:
            minutes = int(minutes)
            if minutes > 0 and minutes < 300:
                return True
        except:
            return False

    runtimeMinutes = input("Please enter the runtime in minutes of your movie:\n")
    while not valid_runtimeMinutes(runtimeMinutes):
        print('Please enter a valid runtime between 0 and 300 minutes.')
        runtimeMinutes = input("Please enter the runtime in minutes of your movie:\n")

    runtimeMinutes = int(runtimeMinutes)

    # genres
    def valid_yn(yn):
        if yn == 'y' or yn =='n':
            return True
        return False

    def request_genre():
        genre = input("Please enter one of your movie genres:\n")
        genre = genre.lower()
        while genre not in valid_genres:
            print('That is not a valid genre.')
            genre = input("Please enter one of your movie genres:\n")
            genre = genre.lower()
        genres.add(genre)
        return

    def request_yn():    
            more = input("Do you want to add more genres to your movie?(y/n)\n")
            while not valid_yn(more):
                more = input("Do you want to add more genres to your movie?(y/n)\n")
            return more


    genres = set()
    request_genre()
    while request_yn() == 'y':
        request_genre()

    # rating
    rating = input("Please enter your movie rating:\n")
    while rating not in valid_ratings:
        print('That is not a valid rating.')
        rating = input("Please enter your movie rating:\n")

    # director
    director = input("Please enter your movie director:\n")
    while director not in valid_directors:
        print('That is not a valid director.')
        director = input("Please enter your movie director:\n")

    # actors
    def request_actor():
        actor = input("Please enter one of your movie actors:\n")
        while actor not in valid_actors:
            print('That is not a valid actor.')
            actor = input("Please enter one of your movie actors:\n")
        actors.add(actor)
        return

    def request_yn_actor():    
            more = input("Do you want to add more actors to your movie?(y/n)\n")
            while not valid_yn(more):
                more = input("Do you want to add more actors to your movie?(y/n)\n")
            return more

    actors = set()
    request_actor()
    while request_yn_actor() == 'y' and len(actors) < 3:
        request_actor()
    if len(actors) == 3:
        print('''Sorry, you can't add any more actors.''')

    story = input("Briefly describe your movie plot (around 150 words).\n")
    return {'story':story,'actors':actors,'director':director,'rating':rating,'genres':genres,'runtimeMinutes':runtimeMinutes,'year':year}

In [15]:
# Run Naive Bayes classifier on story

# Transform the string into a valid model input

# defining necessary funtions
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # turn to lowercase   
    s = s.lower()
    #remove URLs
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    s = re.sub(regex,"",s)
    #replace special characters with spaces
    s = re.sub(r'([^a-zA-Z ]+?)', ' ', s)
    #removes leading and traling whitespaces
    s = s.strip()
    #removes multiple white spaces
    s = re.sub(' +', ' ', s)
    return s

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    #return s.split(" ")
    return nltk.word_tokenize(s)

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist 

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in l]

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    return [word for word in l if not word in stopwords.words()]

def process_text(text):
    '''
    Cleans-up, tokenizes, gets word net position, stems and lemmatizes and then removes the stopwords for a story text.
    Returns the list of resulting 'words'
    '''
    return remove_stopwords(stem_and_lemmatize(tokenize(clean_up(text))))

In [16]:
# I need to import the list of top5000 words
top5000 = pickle.load(open('top5000.pkl',"rb"))

# we need a function that finds the word features (our top most common words)
# in the text
def find_features(text):
    words = set(text)
    features = {}
    for w in top5000:
        features[w] = (w in words)
    return features

In [17]:
# We need to import the classifier we created
classifier = pickle.load(open('NBClassifier.pkl',"rb"))

In [18]:
# make the prediction
def nlp_classifier(story):
    return classifier.classify(find_features(process_text('story')))

In [ ]:
# I will create a table, that contains the stars and directors IMDB and ROI to be able to consult it faster 
# (instead of computing these scores each time)
# Then I'll pickle it and assume it as an input of this notebook

'''
# tables with the average rating of the movies a actor is in, by column
# creating table with actor rating
actors1 = pd.pivot_table(movies, values = 'averageRating', index = 'actor1', aggfunc = 'mean')
actors2 = pd.pivot_table(movies, values = 'averageRating', index = 'actor2', aggfunc = 'mean')
actors3 = pd.pivot_table(movies, values = 'averageRating', index = 'actor3', aggfunc = 'mean')
actors_rating = pd.concat([actors1, actors2, actors3]).groupby(level=0).mean()
actors_rating.reset_index(inplace=True)
actors_rating.drop(actors_rating.index[0],inplace=True)
actors_rating = pd.merge(actors_rating,names,left_on='index',right_on='nconst')
actors_rating.drop(columns='index',inplace=True)
pickle.dump(actors_rating,open('actors_rating.pkl',"wb"))
#creating table with directors rating
directors_rating = pd.pivot_table(movies, values = 'averageRating', index = 'directors', aggfunc='mean')
directors_rating.reset_index(inplace=True)
directors_rating = pd.merge(directors_rating,names,left_on='directors',right_on='nconst')
directors_rating.drop(columns='directors',inplace=True)
pickle.dump(directors_rating,open('directors_rating.pkl',"wb"))

# creating table with actor ROI
actors1 = pd.pivot_table(movies, values = 'ROI', index = 'actor1', aggfunc = 'mean')
actors2 = pd.pivot_table(movies, values = 'ROI', index = 'actor2', aggfunc = 'mean')
actors3 = pd.pivot_table(movies, values = 'ROI', index = 'actor3', aggfunc = 'mean')
actorsROI = pd.concat([actors1, actors2, actors3]).groupby(level=0).mean()
actorsROI.reset_index(inplace=True)
actorsROI = pd.merge(actorsROI,names,left_on='index',right_on='nconst')
actorsROI.drop(columns='index',inplace=True)
pickle.dump(actorsROI,open('actorsROI.pkl',"wb"))
# creating table with director ROI
directorsROI = pd.pivot_table(movies, values = 'ROI', index = 'directors', aggfunc='mean')
directorsROI.reset_index(inplace=True)
directorsROI = pd.merge(directorsROI,names,left_on='directors',right_on='nconst')
directorsROI.drop(columns='directors',inplace=True)
directorsROI.head()
pickle.dump(directorsROI,open('directorsROI.pkl',"wb"))
'''
# since actors can be directors and vice-versa, and to mimic the logic of the 
# data used to build the model I had to keep the tables separate for actors and directors

In [20]:
# loading the previously created tables for ROI and Rating of people
actors_rating = pickle.load(open('actors_rating.pkl',"rb"))
directors_rating = pickle.load(open('directors_rating.pkl',"rb"))
actorsROI = pickle.load(open('actorsROI.pkl',"rb"))
directorsROI = pickle.load(open("directorsROI.pkl","rb"))

In [21]:
# computing starsIMDB, directorIMDB, starsROI, directorROI
def mean_ROI(names):
    '''
    takes a list of actors names and returns the average of average ROIs for the names provided
    '''
    values = []
    for name in names:
        values.append(float(actorsROI[actorsROI.primaryName == name]['ROI']))
    return sum(values)/len(values)

def dir_ROI(director_name):
    '''
    takes the name of a director and returns his/her average ROI
    '''
    return float(directorsROI[directorsROI.primaryName == director_name]['ROI'])

def mean_IMDB(names):
    '''
    takes a list of actors and returns the average of average IMDB ratings 
    for the names provided
    '''
    values = []
    for name in names:
        values.append(float(actors_rating[actors_rating.primaryName == name]['averageRating']))
    return sum(values)/len(values)

def dir_IMDB(director_name):
    '''
    takes the name of a director and returns his/her average IMDB score
    '''
    return float(directors_rating[directors_rating.primaryName == director_name]['averageRating'])


In [22]:
# put all inputs I can on a DF
def process_inputs(raw_inputs):
    '''
    takes a dictionary with user inputs, returns a dataframe ready to be
    used as input for the classifier model
    '''
    # creating empty df:
    inputs = pd.DataFrame(columns = ['startYear','runtimeMinutes','action','adventure','animation','biography','comedy','crime',
                                     'documentary','drama','family','fantasy','history','horror','music','musical','mystery',
                                     'news','romance','sci-fi','sport','thriller','war','western','result','Approved','G',
                                     'NC-17','Not Rated','PG','PG-13','R','TV-14','TV-G','TV-MA','TV-PG','Unrated','X',
                                     'starsIMDB','directorIMDB','starsROI','directorROI','NLPclass'], index =[0])

    # populating a dic that I will later add as a row on the inputs df
    inputs_dic = {}
    inputs_dic['startYear'] = raw_inputs['year']
    inputs_dic['runtimeMinutes'] = raw_inputs['runtimeMinutes']
    for g in valid_genres:
        if g in raw_inputs['genres']: 
            inputs_dic[g] = 1
        else:
            inputs_dic[g] = 0
    for r in valid_ratings:
        if r == raw_inputs['rating']: 
            inputs_dic[r] = 1
        else:
            inputs_dic[r] = 0
    inputs_dic['starsIMDB'] = mean_IMDB(raw_inputs['actors'])
    inputs_dic['directorIMDB'] = dir_IMDB(raw_inputs['director'])
    inputs_dic['starsROI'] = mean_ROI(raw_inputs['actors'])
    inputs_dic['directorROI'] = dir_ROI(raw_inputs['director'])
    inputs_dic['NLPclass'] = nlp_classifier(raw_inputs['story'])

    # creating a datframe and changing dtypes 
    inputs.loc[0] = pd.Series(inputs_dic)
    inputs.drop(columns='result',inplace=True)
    inputs = inputs.astype('float64')
    return inputs

In [23]:
model = pickle.load(open('XGBClassifier.pkl',"rb"))

In [24]:
def predict_outcome(inputs):
    '''
    takes inputs (a single row dataframe) and returns the predicted
    result flop, regular or blockbuster
    '''
    p = int(model.predict(inputs))
    if p == 0:
        return 'Flop'
    if p == 1:
        return 'Regular'
    if p == 2:
        return 'Blockbuster'

In [25]:
predict_outcome(process_inputs(get_inputs()))

Please enter the year your movie will come out :
1999
Please enter the runtime in minutes of your movie:
99
Please enter one of your movie genres:
war
Do you want to add more genres to your movie?(y/n)
ñ
Do you want to add more genres to your movie?(y/n)
n
Please enter your movie rating:
R
Please enter your movie director:
Steven Spielberg
Please enter one of your movie actors:
Tom Hanks
Do you want to add more actors to your movie?(y/n)
n
Briefly describe your movie plot (around 150 words).
A piece of dog poop.


'Blockbuster'